# Practica 2 - Natural Language processing

In [113]:
import os
import pandas as pd

data_dir = "data"
train_path = os.path.join(data_dir, "train.csv")
test_path = os.path.join(data_dir, "test.csv")
df_train = pd.read_csv(train_path, encoding='ISO-8859-1', index_col="textID")
df_test = pd.read_csv(test_path, encoding='ISO-8859-1', index_col="textID")
df_train.head()

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Kmï¿½),Density (P/Kmï¿½)
textID,,,,,,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [114]:
df_train.isna().sum()

text                 1
selected_text        1
sentiment            0
Time of Tweet        0
Age of User          0
Country              0
Population -2020     0
Land Area (Kmï¿½)    0
Density (P/Kmï¿½)    0
dtype: int64

In [115]:
df_test.isna().sum()

text                1281
sentiment           1281
Time of Tweet       1281
Age of User         1281
Country             1281
Population -2020    1281
Land Area (Km²)     1281
Density (P/Km²)     1281
dtype: int64

In [116]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [117]:
df_train["text"].isna().sum()

np.int64(0)

In [118]:
from tensorflow.keras.layers import TextVectorization

vectorize_layer = TextVectorization(
    output_mode='int',
    standardize="lower_and_strip_punctuation",
    split="whitespace",
)

corpus = df_train["text"].values

vectorize_layer.adapt(corpus)

vectorized_train = vectorize_layer(corpus)
vectorized_train

<tf.Tensor: shape=(27480, 33), dtype=int64, numpy=
array([[  293,    17, 15185, ...,     0,     0,     0],
       [  413,   115,     2, ...,     0,     0,     0],
       [    6,  1335,    10, ...,     0,     0,     0],
       ...,
       [  225,    31,    12, ...,     0,     0,     0],
       [   20,     9,    28, ...,     0,     0,     0],
       [   29,    30,  6480, ...,     0,     0,     0]])>

In [119]:
vectorize_layer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('i'),
 np.str_('to'),
 np.str_('the'),
 np.str_('a'),
 np.str_('my'),
 np.str_('and'),
 np.str_('you'),
 np.str_('it')]

In [120]:
import tensorflow as tf
window_size = 2
def generate_skipgram_pairs(sequence):
    # sequence: shape (sentence_length,)
    seq_len = tf.shape(sequence)[0]
    
    # Get all valid center word positions
    i = tf.range(seq_len)
    
    def get_context_pairs(i_t):
        start = tf.maximum(0, i_t - window_size)
        end = tf.minimum(seq_len, i_t + window_size + 1)
        
        center = sequence[i_t]
        # context = sequence[start:i_t] + sequence[i_t+1:end]
        left_context = sequence[start:i_t]
        right_context = sequence[i_t + 1:end]
        context = tf.concat([left_context, right_context], axis=0)
        
        center_tiled = tf.fill([tf.shape(context)[0]], center)
        
        return tf.data.Dataset.from_tensor_slices((center_tiled, context))

    return tf.data.Dataset.from_tensor_slices(i).flat_map(get_context_pairs)


In [121]:
train_ds = tf.data.Dataset.from_tensor_slices(vectorized_train)

vocab_size = len(vectorize_layer.get_vocabulary())
window_size = 3

print(train_ds.element_spec)
train_ds.as_numpy_iterator().next()

TensorSpec(shape=(33,), dtype=tf.int64, name=None)


array([  293,    17, 15185,    69,     2,   120,    47,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0])

In [122]:
train_ds = train_ds.flat_map(generate_skipgram_pairs)
print(train_ds.element_spec)
train_ds.as_numpy_iterator().next()

(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


(np.int64(293), np.int64(17))

In [123]:
embedding_dim = 64

input_target = Input(shape=(vocab_size,), name='target')
input_context = Input(shape=(vocab_size,), name='context')

shared_embedding = Dense(embedding_dim, use_bias=False, name='shared_embedding')

target_embedding = shared_embedding(input_target)
context_embedding = shared_embedding(input_context)

dot_product = Dot(axes=1)([target_embedding, context_embedding])
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(train_ds, epochs=10)

weights = model.get_layer('shared_embedding').get_weights()[0]
print("Matriz de embeddings shape:", weights.shape)

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ target (InputLayer) │ (None, 29164)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context             │ (None, 29164)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_embedding    │ (None, 64)        │  1,866,496 │ target[0][0],     │
│ (Dense)             │                   │            │ context[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_6 (Dot)         │ (None, 1)         │          0 │ shared_embedding… │
│                     │                   │            │ shared_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 1)         │          0 │ dot_6[0][0]       │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,866,496 (7.12 MB)

 Trainable params: 1,866,496 (7.12 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


ValueError: Layer "functional_6" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=() dtype=int64>]

In [125]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import skipgrams

train_ds = tf.data.Dataset.from_tensor_slices(vectorized_train)

vocab_size = len(vectorize_layer.get_vocabulary())
window_size = 3

def tf_skipgrams(sequence):
    sequence = tf.cast(sequence, tf.int32)
    
    def generate_pairs(x):
        pairs, _ = skipgrams(
            x.numpy(),
            vocabulary_size=vocab_size,
            window_size=window_size,
            negative_samples=0  # set to >0 if you want (target, context), label
        )
        if not pairs:
            # Return dummy data if skipgrams returns empty
            return tf.zeros((0, 2), dtype=tf.int32)
        return tf.convert_to_tensor(pairs, dtype=tf.int32)

    pairs_tensor = tf.py_function(func=generate_pairs, inp=[sequence], Tout=tf.int32)
    pairs_tensor.set_shape([None, 2])
    return tf.data.Dataset.from_tensor_slices(pairs_tensor)

train_ds = train_ds.flat_map(tf_skipgrams)

print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

TensorSpec(shape=(2,), dtype=tf.int32, name=None)


array([69, 47], dtype=int32)

In [126]:
train_ds = (
    train_ds
    .map(lambda x: ((tf.one_hot(x[0], depth=vocab_size), tf.one_hot(x[1], depth=vocab_size)), tf.constant(1.0)))
)
print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

((TensorSpec(shape=(29164,), dtype=tf.float32, name=None), TensorSpec(shape=(29164,), dtype=tf.float32, name=None)), TensorSpec(shape=(), dtype=tf.float32, name=None))


((array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
  array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)),
 np.float32(1.0))

In [127]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dot, Activation

# vocab_size = 29164
embedding_dim = 128

# One-hot inputs (shape = (vocab_size,))
input_target = Input(shape=(vocab_size,))
input_context = Input(shape=(vocab_size,))

# Shared embedding layer simulated with Dense layer
embedding_layer = Dense(embedding_dim, use_bias=False)

target_embedding = embedding_layer(input_target)   # shape: (embedding_dim,)
context_embedding = embedding_layer(input_context) # same

# Dot product of embeddings
dot_product = Dot(axes=-1)([target_embedding, context_embedding])

# Sigmoid output for skipgram-style binary prediction
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy')


In [128]:
model.fit(train_ds.batch(128).prefetch(tf.data.AUTOTUNE), epochs=5)


Epoch 1/5
    322/Unknown 18s 53ms/step - loss: 0.6603

KeyboardInterrupt: 